In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
import time
from skimage import io
from skimage.morphology import convex_hull_image
from skimage.measure import label, regionprops
from skimage.color import rgb2gray
from PIL import Image 
import coremltools 
import serial
import os

In [ ]:
ser =serial.Serial('/dev/cu.usbmodem14101',9600)
model = coremltools.models.MLModel('Model/CCNC.mlmodel')
cap=cv2.VideoCapture(1)
needplot=False

In [ ]:
while True:
    #Wait for arduino to write
    waitforreq=ser.read_until()
    #Display serial
    print("serial say : "waitforreq)
    print("Requset code : ",waitforreq[0])
    
    #Check is that really come from arduino
    if (waitforreq[0]!=50):
        continue
    
    #Try capture till got photo
    ret=False
    while (ret==False):
        print("Cam!")
        ret, frame = cap.read() # ret is boolean for pic avalble or not
    
    print("Cam've got shot!")
    
    frame=frame[...,::-1] # cv2 BRG to RGB
    frame=np.rot90(frame, 1, ) # rotate 90 degrees
    frame=frame[:,66:1005] #crop the photo
    
    #Show raw photo
    if needplot:
        plt.imshow(frame)
        plt.show()

    #Calculate binary from R:G:B 1:1:0 and not black pixel R+G+B>700 is black
    binary=((((frame[:,:,0]/frame[:,:,2])+(frame[:,:,1]/frame[:,:,2]))>2.5)&(frame[:,:,0]+frame[:,:,1]+frame[:,:,2]<=700))
    
    #Show first binary
    plt.imshow(binary)
    plt.show()
    
    #Start label first binary
    labelframe=label(binary)
    regiongrframe=regionprops(labelframe)

    print("Start find the nearest chillie.")
    
    labelreal=-1 #lable that we choses defult -1
    yreal=0 #Parameter for dowst chilie
    minr, minc, maxr, maxc=0,0,0,0
    for j in range(len(regiongrframe)):
        area=regiongrframe[j].area
        y0,x0=regiongrframe[j].centroid
        #if area pixel less than 9000 assume as noise
        if area<9000:
            pass
        elif(y0>yreal):
            labelreal= regiongrframe[j].label
            yreal=y0
            minr, minc, maxr, maxc = regiongrframe[j].bbox
    
    #Notting deteccted
    if(labelreal==-1):
        print("-------->no detection")
        command=str.encode('0')
        ser.write(command)
        continue
    
    print("Deleting noise",labelreal)
    #New binary without noise
    binary=(labelframe==labelreal)
    #New frame with Black background
    frame[:,:,0]=frame[:,:,0]*binary
    frame[:,:,1]=frame[:,:,1]*binary
    frame[:,:,2]=frame[:,:,2]*binary
    
    #Crop img
    print("Cropping photo : ",minr,minc, maxr, maxc)
    binary=binary[minr:maxr+1,minc:maxc+1]
    frame=frame[minr:maxr+1,minc:maxc+1,:]

    print("check4")
    R=frame[:,:,0]
    G=frame[:,:,1]
    B=frame[:,:,2]
    #meanofrgb=(R+G+B)/1.65
    
    #biforblen is a binary of Chilie without tail
    print("Detecting tail")
    biforblen=(binary&(R>G*1.8)).copy()
    
    #Show tail (biforblen!=binary)=tail
    if needplot:
        plt.imshow(biforblen!=binary)
        plt.show()
    
    print("tail pixel : ",np.sum(biforblen!=binary))
    
    #if tail pixel less than 2000 mean no tail
    if (np.sum(biforblen!=binary)<2000):
        print("-------->No tail detected")
        #Sent false to arduino
        command=str.encode('0')
        ser.write(command)
        continue
    
    #groupping the real chilie body without tail
    labelframeforblen=label(biforblen)
    regiongrframeforblen=regionprops(labelframeforblen)

    print("Start scope for blen.")
    
    labelrealforblen=-1
    #yrealforblen keep maximum 
    yrealforblen=0
    minr, minc, maxr, maxc=0,0,0,0
    for j in range(len(regiongrframeforblen)):
        area=regiongrframeforblen[j].area
            #y0,x0=regiongrframeforblen[j].centroid
        #if area less that 9000px assume as noise
        if area<9000:
            pass
        elif(area>yrealforblen):
            labelrealforblen= regiongrframeforblen[j].label
            yrealforblen=area
            minr, minc, maxr, maxc = regiongrframeforblen[j].bbox  
    
    #Crop binary for convexhull
    biforblen=biforblen[minr:maxr+1,minc:maxc+1].copy()

    print("Calculate blend using convex" ,biforblen.shape )
    if needplot:
        plt.imshow(biforblen)
        plt.show()
    print("pixel : ",np.sum(biforblen))
    
    #Body of chillie less than 43000 assume as small or green ones
    if (np.sum(biforblen)<=43000):
        print("-------->it green or too small")
        command=str.encode('0')
        ser.write(command)
        continue
    
    #try assing convex hull to cvh
    try :
        cvh=convex_hull_image(biforblen)
    except:
        print("!!!!!!!!!convexhull not working!")
        cvh=biforblen
        
    #show covex hull
    if needplot:
        plt.imshow(cvh)
        plt.show()

    #if binary/convex hull less than 65 assume it is bleded
    if (np.sum(biforblen)*100/np.sum(cvh)<65):
        #return false
        print("-------->No it's blend : ",np.sum(biforblen)*100/np.sum(cvh))
        command=str.encode('0')
        ser.write(command)
        continue
    
    print("Transforme background to white")
    frame[(R+G+B)==0]=255
    img=Image.fromarray(frame)
    print("Calculating colour.")
    
    if needplot:
        plt.imshow(frame)
        plt.show()
    
    #Predict by ML model
    predict=model.predict({'image': img})['classLabel']
    
    #Good chilie detected
    if(predict=='red' or predict=='yellow'):
        #return true
        print("-------->Good chilie")
        command=str.encode('1')
        ser.write(command)
    else:
            #return false
        print("-------->chilie colour not good",predict)
        command=str.encode('0')
        ser.write(command)
    print("donee")